In [1]:
import pandas as pd
import optuna
import yfinance as yf
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pandas_ta as ta
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Nadam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [2]:
import warnings
warnings.filterwarnings('ignore', category = pd.errors.PerformanceWarning)

In [3]:
# Pobieramy dane dotyczące BTC z yahoo
BTC = yf.download('BTC-USD', start = '2022-02-24', end = '2024-01-01')

BTC['Target'] = (BTC['Close'].diff() > 0).astype(int)

def add_lagged_features(data, column_name, n_days):
    for i in range(1, n_days + 1):
        lagged_column_name = f'{column_name}_{i}d_back'
        data[lagged_column_name] = data[column_name].shift(i)
    return data

BTC = add_lagged_features(BTC, 'Close', 30)
BTC = add_lagged_features(BTC, 'Open', 30)
BTC = add_lagged_features(BTC, 'High', 30)
BTC = add_lagged_features(BTC, 'Low', 30)
BTC = add_lagged_features(BTC, 'Volume', 30)

# Usuwamy pierwszych 30 wierszy
BTC = BTC.dropna().reset_index(drop = True)

# print(BTC.head())
# print(len(BTC))
# print(BTC.shape)

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Konwersacja zmiennej Target na zmienną binarną
target = to_categorical(BTC['Target'].values)

# Usuwamy niepotrzebne kolumny
features = BTC.drop(['Target', 'Adj Close'], axis = 1).values 

# Dzielimy dane na treningowe i testowe
split_idx = int(len(features) * 0.8)

m = int(np.floor(0.8 * len(features)))
train_features = features[:m]
test_features = features[m:]
train_target = target[:m]
test_target = target[m:]

In [ ]:
##################################################
##################################################
# WERSJA MAŁO SKOMPLIKOWANA
##################################################
##################################################

In [5]:
# Budujemy model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (train_features.shape[1],)),
    Dense(64, activation = 'relu'),
    Dense(2, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Trenujemy model
history = model.fit(train_features, train_target, epochs = 50, batch_size = 12, validation_split = 0.2)

C:\Users\48798\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.5103 - loss: 3496435968.0000 - val_accuracy: 0.5865 - val_loss: 288644928.0000
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5613 - loss: 426883328.0000 - val_accuracy: 0.5288 - val_loss: 211550496.0000
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5374 - loss: 315929472.0000 - val_accuracy: 0.5673 - val_loss: 168483008.0000
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5814 - loss: 262845424.0000 - val_accuracy: 0.5288 - val_loss: 158888864.0000
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6128 - loss: 198717968.0000 - val_accuracy: 0.5000 - val_loss: 200619664.0000
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5793 - loss: 184670800.0000 - val_accuracy: 0.5288 - val_loss: 134805200.0000
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6438 - loss: 174883104.0000 - val_accuracy: 0.5192 - val_loss: 214497872.0000
Epo

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6968 - loss: 24909342.0000 - val_accuracy: 0.4519 - val_loss: 47837360.0000
Epoch 48/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7689 - loss: 19634026.0000 - val_accuracy: 0.5096 - val_loss: 48001320.0000
Epoch 49/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7419 - loss: 20652634.0000 - val_accuracy: 0.5192 - val_loss: 51953616.0000
Epoch 50/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7550 - loss: 15652956.0000 - val_accuracy: 0.5000 - val_loss: 46908880.0000


In [6]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
186
199
75
56
Accuracy (ACC): 0.7461240310077519
True Positive Rate (TPR): 0.768595041322314
False Positive Rate (FPR): 0.2737226277372263
True Negative Rate (TNR): 0.7262773722627737
Positive Predictive Value (PPV): 0.7126436781609196
Negative Predictive Value (NPV): 0.7803921568627451


In [7]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
34
28
34
34
Accuracy (ACC): 0.47692307692307695
True Positive Rate (TPR): 0.5
False Positive Rate (FPR): 0.5483870967741935
True Negative Rate (TNR): 0.45161290322580644
Positive Predictive Value (PPV): 0.5
Negative Predictive Value (NPV): 0.45161290322580644


In [ ]:
##################################################
##################################################
##################################################
##################################################

In [ ]:
##################################################
##################################################
# WERSJA BARDZIEJ SKOMPLIKOWANA
##################################################
##################################################

In [ ]:
# Korzystamy z biblioteki OPTUNA, która służy do automatycznego dostrajania hiperparametrów
def create_model(trial):
    # Liczba warstw ukrytych, od 1 do 3
    n_layers = trial.suggest_int('n_layers', 1, 3)
    model = Sequential()
    # Dodanie pierwszej warstwy ukrytej
    model.add(Dense(trial.suggest_int('n_units_first', 10, 300), activation = 'relu', input_shape = (train_features.shape[1],)))
    # Dodanie warstwy Dropout
    model.add(Dropout(trial.suggest_float('dropout_first', 0.0, 0.5)))

    # Dodawanie kolejnych warstw ukrytych w pętli zależnie od liczby warstw
    for i in range(n_layers):
        model.add(Dense(trial.suggest_int(f'n_units_{i}', 10, 300), activation = 'relu'))
        model.add(Dropout(trial.suggest_float(f'dropout_{i}', 0.0, 0.5)))

    # Dodanie warstwy wyjściowej
    model.add(Dense(2, activation = 'softmax'))
    # Sugestia Optuna dotycząca współczynnika uczenia dla optymalizatora Adama
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(optimizer = Adam(learning_rate = lr), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

# Definicja funkcji celu dla procesu dostrajania Optuna
def objective(trial):
    # Tworzenie modelu z bieżącymi parametrami sugerowanymi przez Optuna
    model = create_model(trial)
    # Ustawienie mechanizmu wczesnego zatrzymywania treningu w celu uniknięcia przeuczenia
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1, restore_best_weights = True)
    model.fit(train_features, train_target, epochs = 50, batch_size = 32, validation_split = 0.2, 
              callbacks = [early_stopping], verbose = 0)
    _, accuracy = model.evaluate(test_features, test_target, verbose = 0)
    return accuracy

# Uruchomienie procesu dostrajania
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10)

# Wyświetlenie najlepszych parametrów znalezionych przez Optuna
print('Najlepsze parametry:', study.best_trial.params)

In [ ]:
best_params = {'n_layers': 2, 
               'n_units_first': 210, 
               'dropout_first': 0.3128939094692668, 
               'n_units_0': 196, 
               'dropout_0': 0.3672186353981186, 
               'n_units_1': 101, 
               'dropout_1': 0.32781184562902965, 
               'lr': 4.665241600838611e-05}

# Budujemy model sieci neuronowej
model = Sequential()
model.add(Dense(best_params['n_units_first'], activation = 'relu', input_shape = (train_features.shape[1],)))
model.add(Dropout(best_params['dropout_first']))

model.add(Dense(best_params['n_units_0'], activation = 'relu'))
model.add(Dropout(best_params['dropout_0']))

model.add(Dense(best_params['n_units_1'], activation = 'relu'))
model.add(Dropout(best_params['dropout_1']))

#model.add(Dense(best_params['n_units_2'], activation = 'relu'))
#model.add(Dropout(best_params['dropout_2']))

model.add(Dense(2, activation = 'softmax'))

# Optymalizator Adam
optimizer = Adam(learning_rate = best_params['lr'])

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Dodajemy funkcję, która przerwie trenowanie modelu, gdy model przestaje poprawiać swoją wydajność na zbiorze walidacyjnym
early_stopping = EarlyStopping(monitor = 'val_loss',  
                               patience = 10,         # liczba epok bez poprawy po której trening zostanie zatrzymany
                               verbose = 1,           
                               restore_best_weights = True) 


# Trenowanie modelu z dodaniem callbacku
history = model.fit(train_features, train_target, 
                    epochs = 100, 
                    batch_size = 32, 
                    validation_split = 0.2,
                    callbacks = [early_stopping]) 

# Ocena modelu na danych testowych
test_performance = model.evaluate(test_features, test_target)
print('Test loss:', test_performance[0])
print('Test accuracy:', test_performance[1])

# Ocena modelu na danych treningowych
train_performance = model.evaluate(train_features, train_target)
print('Train loss:', train_performance[0])
print('Train accuracy:', train_performance[1])

In [ ]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

In [ ]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)